# Example Notebook for Testing pyBSM Perturbers
You can use this notebook to run an example image through a series of perturbations as defined by the parameters provided to the pybsm package. To use this notebook, install all the dependencies as described in the [README](../README.md).

## Table of Contents
* [Set Up the Environment](#environment-setup)
* [Define PybsmSensor Parameters](#pybsmsensor-params)
* [Define PybsmScenario Parameters](#pybsmscenario-params)

To run this notebook in Colab, use the link below:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Kitware/nrtk/blob/main/examples/pybsm/pybsm_test.ipynb)

## Set Up the Environment <a name="environment-setup"></a>

**Note for Colab users**: After setting up the environment, you may need to "Restart Runtime" in order to resolve package version conflicts (see the [README](../README.md) for more info).

Note: We are suppressing warnings within this notebook to reduce visual clutter for demonstration purposes. If any issues arise while executing this notebook, we recommend that this cell is **not** executed so that any related warnings are shown. 

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys

!{sys.executable} -m pip install -qU pip
print("Installing nrtk...")
!{sys.executable} -m pip install -q nrtk
print("Installing headless OpenCV...")
!{sys.executable} -m pip uninstall -qy opencv-python opencv-python-headless  # make sure they're both gone.
!{sys.executable} -m pip install -q opencv-python-headless

Installing nrtk...


**PyBSM Generator will be added once it is integrated the full pertubation pipeline functionality will be added**

In [3]:
import os
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
import pybsm
import copy
from nrtk.impls.perturb_image.pybsm.sensor import PybsmSensor
from nrtk.impls.perturb_image.pybsm.scenario import PybsmScenario

## Define PybsmSensor Parameters <a name="pybsmsensor-params"></a>

First, we define the PybsmSensor parameters. These parameters describe the sensor that was used to capture the original image.

In [4]:
#dirpath = os.path.dirname(os.path.abspath(__file__))
data_dir = './data'
os.makedirs(data_dir, exist_ok=True)

url = 'https://data.kitware.com/api/v1/item/6596fde89c30d6f4e17c9efc/download'

img_path = os.path.join(data_dir, 'M-41 Walker Bulldog (USA) width 319cm height 272cm.tiff')

if not os.path.isfile(img_path):
    _ = urllib.request.urlretrieve(url, img_path)

imgfile = './data/M-41 Walker Bulldog (USA) width 319cm height 272cm.tiff'

name = 'L32511x'

#telescope focal length (m)
f=4                     
# Telescope diameter (m)
D=275e-3               

#detector pitch (m)
p=.008e-3    
       
#Optical system transmission, red  band first (m)
optTransWavelengths = np.array([0.58-.08,0.58+.08])*1.0e-6
#guess at the full system optical transmission (excluding obscuration)
opticsTransmission=0.5*np.ones(optTransWavelengths.shape[0]) 

# Relative linear telescope obscuration
eta=0.4 #guess     

#detector width is assumed to be equal to the pitch
wx=p                    
wy=p 
#integration time (s) - this is a maximum, the actual integration time will be
#determined by the well fill percentage
intTime=30.0e-3 

#dark current density of 1 nA/cm2 guess, guess mid range for a silicon camera
darkCurrent = pybsm.darkCurrentFromDensity(1e-5,wx,wy)

#rms read noise (rms electrons)
readNoise=25.0 

#maximum ADC level (electrons)
maxN=96000.0

#bit depth
bitdepth=11.9

#maximum allowable well fill (see the paper for the logic behind this)
maxWellFill = .6

#jitter (radians) - The Olson paper says that its "good" so we'll guess 1/4 ifov rms
sx = 0.25*p/f
sy = sx

#drift (radians/s) - again, we'll guess that it's really good
dax = 100e-6
day = dax

#etector quantum efficiency as a function of wavelength (microns) 
#for a generic high quality back-illuminated silicon array
# https://www.photometrics.com/resources/learningzone/quantumefficiency.php
qewavelengths=np.array([.3, .4, .5, .6, .7, .8, .9, 1.0, 1.1])*1.0e-6
qe=np.array([0.05, 0.6, 0.75, 0.85, .85, .75, .5, .2, 0])

sensor = PybsmSensor(name, D, f, p, optTransWavelengths,
                                opticsTransmission, eta, wx, wy,
                                intTime, darkCurrent, readNoise,
                                maxN, bitdepth, maxWellFill, sx, sy,
                                dax, day, qewavelengths, qe)

## Define PybsmScenario Parameters <a name="pybsmscenario-params"></a>

Next, we define the PybsmScenario parameters. These parameters relate to the environmental factors that are present in the image and affect the image quality.

In [5]:
#sensor altitude
altitude=9000.0
#range to target
groundRange = 60000.0

scenario_name = 'niceday'
#weather model
ihaze = 1

scenario = PybsmScenario(scenario_name,ihaze,altitude,groundRange)
scenario.aircraftSpeed = 100.0 #ground speed in m/s